In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [3]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiple springs/dampers

In [4]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

add_coordinate!(vms, ConstCoord(1.0); id="joint target angle")
add_coordinate!(vms, CoordDifference(".robot.rh_LFJ1_coord", "joint target angle"); id="joint angle error")
add_component!(vms, LinearSpring(0.1, "joint angle error"); id="joint spring")
add_component!(vms, LinearDamper(0.01, "joint angle error"); id="joint damper")

"joint damper"

## Simulating the Robot

### Setting Up the Simulation

In [5]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the lateral pinch motion while 1 being 
the end
"""
function update_power_sphere_coord(args, cache, coord)
    nothing
end

function f_setup(cache) 
    nothing
    # return  (get_compiled_coordID(cache, "angular spring length"), get_compiled_coordID(cache, "th spring length"), get_compiled_coordID(cache, "J3 target angle"),
    #         get_compiled_coordID(cache, "J2 target angle"), get_compiled_coordID(cache, "J1 target angle"), get_compiled_coordID(cache, "th J5 target angle"),
    #         get_compiled_coordID(cache, "th J2 target angle"), get_compiled_coordID(cache, "th J1 target angle"))
end

function f_control(cache, t, args, extra)

    # t_start = 0.
    # t_end = 4. #we want to reach the end position at t = t_end 
    
    # if t >= t_start && t <= t_end 
    #     coord_value = (t - t_start) /(t_end -t_start)
    #     update_power_sphere_coord(args, cache, coord_value)
    # end
    nothing
end

tspan = (0., 5.)
vms_compiled = compile(vms)

q_init = generate_q_init(vms_compiled)
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot to assess computational performances"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-5, reltol=1e-5);

#######   Tsit5 , 5 seconds, tol 1e-5, spring stiffness 10: #######
# THJ5 : 1.26   # LFJ5 : 0.45
# THJ4 : 0.26   # LFJ4 : 0.43
# THJ3 : 2.56   # LFJ3 : 1.30
# THJ2 : 3.35   # LFJ2 : 7.1
# THJ1 : 13.51  # LFJ1 : 33.35

# Rosenbrock23 : 2.58
# Rodas5P : 3.21
# TRBDF2 : > 1h
# QNDF : > 1h
# FBDF : > 1h
# Kvaerno5 : > 1h
# KenCarp4 : > 1h

[ Info: Simulating shadow robot to assess computational performances
ODE   0%|█                                              |  ETA: N/A
ODE   3%|██                                             |  ETA: 0:01:00
ODE   7%|████                                           |  ETA: 0:00:37
ODE  12%|██████                                         |  ETA: 0:00:29
ODE  16%|████████                                       |  ETA: 0:00:25
ODE  21%|██████████                                     |  ETA: 0:00:22
ODE  26%|█████████████                                  |  ETA: 0:00:19
ODE  31%|███████████████                                |  ETA: 0:00:17
ODE  35%|█████████████████                              |  ETA: 0:00:16
ODE  40%|███████████████████                            |  ETA: 0:00:14
ODE  45%|█████████████████████                          |  ETA: 0:00:13
ODE  49%|████████████████████████                       |  ETA: 0:00:11
ODE  54%|██████████████████████████                     |  ETA: 0:00:10

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 21714-element Vector{Float64}:
 0.0
 1.6360539262113346e-5
 2.3887154504862646e-5
 3.6540136501569995e-5
 4.7796728349759636e-5
 6.267241705189841e-5
 7.857234316081845e-5
 9.728893177378924e-5
 0.00011803435375111758
 0.00014156593216541259
 ⋮
 4.998209503474172
 4.99844536953393
 4.998681235584315
 4.998917101609707
 4.999152967625727
 4.999388833663615
 4.999624699692132
 4.999860565742517
 5.0
u: 21714-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [-9.446459563202131e-9, 1.9237855917334742e-6, -2.8426306429342288e-8, -2.3286983748273364e-5, -4.725238222865009e-5, 0.0003889421851784235, -2.9014919422833966e-8, -2.3478734888658337e-5, -4.70251500557681e-5, 0.00038890964414251987  …  2.471742044640793, 0.003419350474035966, -6.692773011970704, -1.7051813676674208, 43.703052715707365, 0.16544049908584998, 1.1730815777

### Visualizing the Results

In [29]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_tests.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);